卷积神经网络讲解--以MNIST数据集为例

卷积神经网络与传统神经网络的区别在于其的卷积层以及池化层，主要的目的就是生成一个图像的局部特征

In [37]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./data/MNIST_data",one_hot=True)
sess=tf.InteractiveSession()

Extracting ./data/MNIST_data\train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data\t10k-labels-idx1-ubyte.gz


C:\Users\SmallDragon\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [38]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


我们需要定义权重以及偏置的初始设定函数，方便后期不断地卷积

In [39]:
def WeightInitialized(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)#初始随机卷积核
    return (tf.Variable(initial))

In [40]:
def BiasInitialized(shape):
    initial=tf.constant(0.1,shape=shape)#初始化偏置
    return (tf.Variable(initial))

In [41]:
#卷积设计
def Conv(x,W):
    return(tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME'))

In [42]:
#池化层设计
def Pooling(x):
    return(tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME"))

In [43]:
#load data 先使用占位符 特征分别是input 784维 output one hot 10维
x=tf.placeholder("float",shape=[None,784])
y=tf.placeholder("float",shape=[None,10])

设计网络

In [44]:
#将图像reshaoe
x_image=tf.reshape(x,[-1,28,28,1])

In [45]:
#第一层卷积
W_conv1=WeightInitialized([5,5,1,32])#设计卷积核 5*5的patch 32 个卷积核 
b_conv1=BiasInitialized([32])
h_conv1=tf.nn.relu(Conv(x_image,W_conv1)+b_conv1)
h_pool1=Pooling(h_conv1)

In [46]:
#第二层卷积
W_conv2=WeightInitialized([5,5,32,64])#设计卷积核 5*5的patch 64 个卷积核 
b_conv2=BiasInitialized([64])
h_conv2=tf.nn.relu(Conv(h_pool1,W_conv2)+b_conv2)
h_pool2=Pooling(h_conv2)

In [47]:
#全连接
W_fullConnected=WeightInitialized([7*7*64,1024])#全连接权重矩阵
b_fullConnected=BiasInitialized([1024])#偏置单元
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])#将矩阵扁平化
#前向传播第一层
h_fullContected=tf.nn.relu(tf.matmul(h_pool2_flat,W_fullConnected)+b_fullConnected)

In [48]:
#drop out 层
keep_prob=tf.placeholder("float")
h_fullContected_drop=tf.nn.dropout(h_fullContected,keep_prob)#x,keep_drop 设置神经元被选中的概率

In [49]:
#前向传播第二层 输出结果 softmax
W_fullConnected2=WeightInitialized([1024,10])
b_fullConnected2=BiasInitialized([10])
y_=tf.nn.softmax(tf.matmul(h_fullContected_drop,W_fullConnected2)+b_fullConnected2)

现在完成了全部的前向传播设计

In [51]:
#损失函数 交叉熵设计
cross_entropy=-tf.reduce_sum(y*tf.log(y_))
#梯度下降训练 使损失函数最小
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,"float"))
sess.run(tf.initialize_all_variables())
for i in range(20000):
    batch=mnist.train.next_batch(50)
    if(i%100==0):
        train_accuracy=accuracy.eval(feed_dict={x:batch[0],y:batch[1],keep_prob:1.0})
        print("step {0},training accuracy {1}".format(i,train_accuracy))
    train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

step 0,training accuracy 0.07999999821186066
step 100,training accuracy 0.800000011920929
step 200,training accuracy 0.9399999976158142
step 300,training accuracy 0.8799999952316284
step 400,training accuracy 0.9599999785423279
step 500,training accuracy 0.9800000190734863
step 600,training accuracy 0.9200000166893005
step 700,training accuracy 0.9599999785423279
step 800,training accuracy 0.9399999976158142
step 900,training accuracy 1.0
step 1000,training accuracy 0.9599999785423279
step 1100,training accuracy 0.9399999976158142
step 1200,training accuracy 0.9399999976158142
step 1300,training accuracy 0.9800000190734863
step 1400,training accuracy 1.0
step 1500,training accuracy 0.9800000190734863
step 1600,training accuracy 0.9399999976158142
step 1700,training accuracy 0.9800000190734863
step 1800,training accuracy 0.9800000190734863
step 1900,training accuracy 0.9599999785423279
step 2000,training accuracy 0.9599999785423279
step 2100,training accuracy 0.9800000190734863
step 220

ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: Conv2D_5 = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape_4, Variable_18/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: add_12/_31 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_48_add_12", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current a